## Predicting Price with Neighborhood

**Project Goal: Building on ```1-price-and-location```. The aim is to create a model that can predict the price of apartments in the city of Buenos Aires - with a focus on apartments that cost less than $400,000 USD. The model uses location variable: Neighborhoo**

In [2]:
import warnings
from glob import glob

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
from category_encoders import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge 
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted


warnings.simplefilter(action="ignore", category=FutureWarning)

### 1. Prepare Data

#### Import

#### Explore

#### Split

### 2. Build Model

#### Baseline

#### Iterate

#### Evaluate

### 3. Communicate Results